In [1]:
from pycparser import c_parser, c_ast

In [3]:
import os
orig_path = '/home/maciej/ftb/kernels_lore/orig/'

for filename in os.listdir(orig_path):
    if filename.endswith(".c"): 
        print(os.path.join(orig_path, filename))
    else:
        continue

/home/maciej/ftb/kernels_lore/orig/lore_23155c0c-6959-43e3-957b-9e85593448bd_1217.c
/home/maciej/ftb/kernels_lore/orig/lore_30d681f7-f3ac-4484-8cb1-d0a88edb4d0c_281.c
/home/maciej/ftb/kernels_lore/orig/lore_74df51a8-f031-493e-8814-af9fb2bb2cf7_1087.c
/home/maciej/ftb/kernels_lore/orig/lore_50b8bfaf-8ba7-493e-a226-8d2ec67d0078_229.c
/home/maciej/ftb/kernels_lore/orig/lore_301d9585-272e-4ae4-b0df-f34f93841fae_245.c
/home/maciej/ftb/kernels_lore/orig/lore_1f71e097-a54f-4e21-aa18-60c01ec75f66_355.c
/home/maciej/ftb/kernels_lore/orig/lore_a80d64d6-a725-458a-95bf-7bf94361b0ea_3751.c
/home/maciej/ftb/kernels_lore/orig/lore_5d5689b3-cca7-48f1-93ae-d582b66c1cbb_4061.c
/home/maciej/ftb/kernels_lore/orig/lore_f3a3a269-a04c-4cfa-8c64-7c6fe2a3625d_215.c
/home/maciej/ftb/kernels_lore/orig/lore_02f8f105-b887-414c-aeaf-335111c3e600_3037.c
/home/maciej/ftb/kernels_lore/orig/lore_3a9b6142-ebfa-46bb-b03a-b90dd3063234_268.c
/home/maciej/ftb/kernels_lore/orig/lore_50b8bfaf-8ba7-493e-a226-8d2ec67d0078_1677.

/home/maciej/ftb/kernels_lore/orig/lore_23155c0c-6959-43e3-957b-9e85593448bd_270.c
/home/maciej/ftb/kernels_lore/orig/lore_d7e46df7-7bec-43eb-bf80-be027ee95366_93.c
/home/maciej/ftb/kernels_lore/orig/lore_1e5201b1-b9b4-442e-960e-94b11bbc01bc_265.c
/home/maciej/ftb/kernels_lore/orig/lore_2ff98fa2-83b6-4295-a0da-b7824cb706bb_214.c
/home/maciej/ftb/kernels_lore/orig/lore_b0ae4577-72b2-443a-aa4b-bc72c86cb93d_294.c
/home/maciej/ftb/kernels_lore/orig/lore_30d681f7-f3ac-4484-8cb1-d0a88edb4d0c_1630.c
/home/maciej/ftb/kernels_lore/orig/lore_26166dea-4a8b-479a-a5d8-04b381eb077a_103.c
/home/maciej/ftb/kernels_lore/orig/lore_b919fe3c-b703-4275-af94-4af917a18441_334.c
/home/maciej/ftb/kernels_lore/orig/lore_8437441e-5a5d-4f08-b03a-e308d57885a7_1051.c
/home/maciej/ftb/kernels_lore/orig/lore_23155c0c-6959-43e3-957b-9e85593448bd_3601.c
/home/maciej/ftb/kernels_lore/orig/lore_62b114a7-0f39-4233-9e03-68f7a1a0933f_713.c
/home/maciej/ftb/kernels_lore/orig/lore_2b575b90-58f1-46a3-a7c6-5fe19f136533_113.c
/h

In [3]:
code = """
typedef struct
{
    double alpha;
    double **log_prob_w;
    int num_topics;
    int num_terms;
} lda_model;"""
    
parser = c_parser.CParser()
ast = parser.parse(code)

In [11]:
ast.ext[0].type.type

In [152]:
def max_set(set):
    return set.pop if len(set) == 1 else 'max(' + ','.join(set) + ')'


def q(n, maxs={}):
    if type(n) is str:
        return max_set(maxs[n]) if n in maxs else n
    if type(n) is c_ast.ID:
        return list(maxs[n.name])[0] if n.name in maxs else n.name
    elif type(n) is c_ast.BinaryOp:
        return q(n.left, maxs) + n.op + q(n.right, maxs)
    elif type(n) is c_ast.Constant:
        return n.value
    else:
        return '42'
    
    

class ArrayRefVisitor(c_ast.NodeVisitor):
    def __init__(self, maxs):
        self.refs = {}
        self.maxs = maxs
        
    def visit_ArrayRef(self, node):
        n = node.name.name
        s = node.subscript
        
        if n in self.refs:
            self.refs[n].add(q(s, self.maxs))
        else:
            self.refs[n] = set([q(s, self.maxs)])
            
            
class AssignmentVisitor(c_ast.NodeVisitor):
    def __init__(self, maxs):
        self.refs = {}
        self.maxs = maxs
        
    def visit_Assignment(self, node):
        if node.op == '=':
            l = node.lvalue
            r = node.rvalue

            if type(l) is c_ast.ID:
                if l.name in self.maxs:
                    self.maxs[l.name].add(q(r, self.maxs))
                else:
                    self.maxs[l.name] = set([q(r, self.maxs)])
            

class ForVisitor(c_ast.NodeVisitor):
    def __init__(self):
        self.maxs = {}
            
    def visit_For(self, node):
        c = node.cond
        if type(c) is not c_ast.BinaryOp:
            return
        
        v = c.left
        m = c.right
        
        if type(v) is not c_ast.ID:
            return
        
        if v.name in self.maxs:
            self.maxs[v.name].add(q(m))
        else:
            self.maxs[v.name] = set([q(m)])
            
        c_ast.NodeVisitor.generic_visit(self, node)
        
        
cv = ForVisitor()
cv.visit(ast)    
print(cv.maxs)
        
cv = AssignmentVisitor(cv.maxs)
cv.visit(ast)
print(cv.maxs)
        
cv = ArrayRefVisitor(cv.maxs)
cv.visit(ast)
print(cv.refs)
        

{'i': {'n'}, 'j': {'n'}}
{'i': {'n', '0'}, 'j': {'n', '0'}, 'q': {'n'}}
{'A': {'n*n+n'}, 'D': {'n'}}


In [147]:
q('q+1', cv.maxs)

'q+1'

In [125]:
ast.show()

FileAST: 
  Typedef: long_long, [], ['typedef']
    TypeDecl: long_long, []
      IdentifierType: ['double']
  Decl: i, [], [], []
    TypeDecl: i, []
      IdentifierType: ['int']
  Decl: n, [], [], []
    TypeDecl: n, []
      IdentifierType: ['int']
  Decl: j, [], [], []
    TypeDecl: j, []
      IdentifierType: ['int']
  Decl: q, [], [], []
    TypeDecl: q, []
      IdentifierType: ['int']
  Decl: A, [], [], []
    PtrDecl: []
      TypeDecl: A, []
        IdentifierType: ['double']
  Decl: D, [], [], []
    PtrDecl: []
      TypeDecl: D, []
        IdentifierType: ['double']
  FuncDef: 
    Decl: loop, [], [], []
      FuncDecl: 
        ParamList: 
          Decl: set, [], [], []
            TypeDecl: set, []
              IdentifierType: ['int']
          Decl: values, [], [], []
            PtrDecl: []
              TypeDecl: values, []
                IdentifierType: ['long_long']
        TypeDecl: loop, []
          IdentifierType: ['int']
    Compound: 
      Pragma: scop
  

In [150]:
len(set([3]))

1